In [318]:

import pandas as pd
import numpy as np
from scipy.stats import spearmanr, shapiro, pearsonr, ttest_ind, mannwhitneyu, chi2_contingency, kruskal, f_oneway
from sklearn.preprocessing import LabelEncoder

In [319]:

df = pd.read_csv('corrected_data.csv', delimiter=',') 
df['год'] = df['год'].astype(str)
df['countmen'] = 1
# Столбец countmen для будущего подсчета людей в каждой группе
df.head()


,Unnamed: 0,год,время_года,климат,город,страна,способ_охлаждения,режим_при_смешанном_типе_охлаждения,способ_обогрева,возраст,...,скорость_воздуха,рост,вес,занавески,вентилятор,окно,двери,среднемесячная_температура_на_улице,количество_рекламаций,countmen
0,0,2011.0,Лето,Cубтропический океанический,Техас,США,Кондиционирование,нет,нет,22,...,0.26,170.2,63.0,1.0,1.0,0.0,0.0,28.9,0,1
1,2,2011.0,Лето,Cубтропический океанический,Техас,США,Кондиционирование,нет,нет,42,...,0.08,170.2,63.0,0.0,0.0,0.0,0.0,32.8,0,1
2,5,2011.0,Лето,Cубтропический океанический,Техас,США,Кондиционирование,нет,нет,27,...,0.21,170.2,63.0,1.0,1.0,0.0,0.0,28.9,0,1
3,6,2011.0,Лето,Cубтропический океанический,Техас,США,Кондиционирование,нет,нет,32,...,0.10,170.2,63.0,1.0,1.0,0.0,0.0,32.8,0,1
4,8,2011.0,Лето,Cубтропический океанический,Техас,США,Кондиционирование,нет,нет,43,...,0.15,170.2,63.0,1.0,1.0,0.0,0.0,32.8,0,1


In [320]:
df.isna().sum()

Unnamed: 0                                       0
год                                              0
время_года                                       0
климат                                           0
город                                            0
страна                                           0
способ_охлаждения                                0
режим_при_смешанном_типе_охлаждения              0
способ_обогрева                                  0
возраст                                          0
пол                                              0
ощущение_температуры                             0
ощущение_температуры_(bool)                      0
предпочтительное_изменение_температуры           0
ощущение_движения_воздуха_(bool)               258
предпочтительное_изменение_движения_воздуха      0
оценка_комфорта                                268
утепление                                        0
температура_воздуха_в_помещении                  0
температура_воздуха_на_улице   

In [321]:
# Шкалы всех столбцов для последующей корреляции
types = {
    'год': 'cat',
    'город': 'cat',
    'страна': 'cat',
    'возраст': 'num',
    'пол': 'cat',    
    'утепление':'num',    
    'количество_рекламаций': 'num',
    'время_года': 'cat', 
    'климат': 'cat', 
    'способ_охлаждения': 'cat', 
    'режим_при_смешанном_типе_охлаждения': 'cat', 
    'способ_обогрева': 'cat', 
    'температура_воздуха_в_помещении': 'num',     
    'rh': 'num', 
    'скорость_воздуха': 'num', 
    'занавески': 'cat',
    'вентилятор': 'num',
    'окно': 'cat', 
    'двери': 'cat', 
    'среднемесячная_температура_на_улице': 'num'}
subs = {'ощущение_температуры': 'cat',        
    'ощущение_температуры_(bool)': 'cat',
    'предпочтительное_изменение_температуры': 'cat',
    'ощущение_движения_воздуха_(bool)': 'cat',
    'предпочтительное_изменение_движения_воздуха': 'cat',
    'оценка_комфорта': 'num'}

# Создание функций для добавления категориальных столбцов датафрейма
def changerecl(x):
    if x <= 1:
        return 'мало'
    if x == 2:
        return 'средне'
    if x > 2:
        return 'много'
    
def changeage(x):
    if x <= 44:
        return 'молодой возраст'
    if 45 <= x <= 59:
        return 'средний возраст'
    if x >= 60:
        return 'пожилой возраст'
def standr_comf_rh(df):
    if 40 <= df['rh'] <= 60:
        return 'OK'
    else:
        return 'not OK'
# Функции для выполнения корреляций

def hi2(g1, g2):
    
    ct = pd.crosstab(g1, g2)
    chi2 = chi2_contingency(ct)
    return (chi2.statistic / (df.shape[0] * (min(df.shape) - 1))) ** 0.5, chi2.pvalue

def corr(g1, g2, type1, type2):
    res = 0
    method = ''
    if type1 != type2:
        norm = 0
        if type1 == 'cat':
            norm = shapiro(g2).pvalue >= 0.05
            gs = [g2[g1 == i] for i in g1.unique()]
            ln = len(g1.unique())
        else:
            gs = [g1[g2 == i] for i in g2.unique()]
            norm = shapiro(g1).pvalue >= 0.05
            ln = len(g2.unique())
        if not norm:
            if ln == 2:

                res = mannwhitneyu(*gs)
                method = 'Mann-Whitneyu'
            else:
                res = kruskal(*gs)
                method = 'Kruskal-Wallis'
        else: 
            if ln == 2:
                res = ttest_ind(*gs)
                method = 'Student'
            else:
                res = f_oneway(*gs)
                method = 'ANOVA'
    else:
        if type1 == 'cat':
            method = 'Chi2'
            res = hi2(g1, g2)
            
        else:
            norm = shapiro(g1).pvalue >= 0.05 and shapiro(g2).pvalue >= 0.05
            if not norm:
                res = spearmanr(g1, g2)
                method = 'Spearman'
            else:
                res = pearsonr(g1, g2)
                method = 'Pearson'
    return res[0], res[1], method



In [322]:

# Категоризация требуемых данных в отдельных столбцах
df['реклам_категория'] = df['количество_рекламаций'].map(changerecl)
df['возрастная_категория'] = df['возраст'].map(changeage)
df.head()

,Unnamed: 0,год,время_года,климат,город,страна,способ_охлаждения,режим_при_смешанном_типе_охлаждения,способ_обогрева,возраст,...,вес,занавески,вентилятор,окно,двери,среднемесячная_температура_на_улице,количество_рекламаций,countmen,реклам_категория,возрастная_категория
0,0,2011.0,Лето,Cубтропический океанический,Техас,США,Кондиционирование,нет,нет,22,...,63.0,1.0,1.0,0.0,0.0,28.9,0,1,мало,молодой возраст
1,2,2011.0,Лето,Cубтропический океанический,Техас,США,Кондиционирование,нет,нет,42,...,63.0,0.0,0.0,0.0,0.0,32.8,0,1,мало,молодой возраст
2,5,2011.0,Лето,Cубтропический океанический,Техас,США,Кондиционирование,нет,нет,27,...,63.0,1.0,1.0,0.0,0.0,28.9,0,1,мало,молодой возраст
3,6,2011.0,Лето,Cубтропический океанический,Техас,США,Кондиционирование,нет,нет,32,...,63.0,1.0,1.0,0.0,0.0,32.8,0,1,мало,молодой возраст
4,8,2011.0,Лето,Cубтропический океанический,Техас,США,Кондиционирование,нет,нет,43,...,63.0,1.0,1.0,0.0,0.0,32.8,0,1,мало,молодой возраст


In [323]:
dfmeanage = df[['пол', 'страна', 'возраст']].groupby(['пол', 'страна']).mean().round().astype(int)
dfmeanage.head()
# Рассчитать средний возраст респондентов по полу и стране

возраст
пол     страна            
Женский Австралия       46
        Индия           35
        США             38
Мужской Австралия       43
        Индия           37

In [324]:
goodtemp = df.loc[df['предпочтительное_изменение_температуры'] == 'Без изменений']
goodtemp.head()
# Сбор всех данных с комфортной температурой

,Unnamed: 0,год,время_года,климат,город,страна,способ_охлаждения,режим_при_смешанном_типе_охлаждения,способ_обогрева,возраст,...,вес,занавески,вентилятор,окно,двери,среднемесячная_температура_на_улице,количество_рекламаций,countmen,реклам_категория,возрастная_категория
0,0,2011.0,Лето,Cубтропический океанический,Техас,США,Кондиционирование,нет,нет,22,...,63.0,1.0,1.0,0.0,0.0,28.9,0,1,мало,молодой возраст
1,2,2011.0,Лето,Cубтропический океанический,Техас,США,Кондиционирование,нет,нет,42,...,63.0,0.0,0.0,0.0,0.0,32.8,0,1,мало,молодой возраст
2,5,2011.0,Лето,Cубтропический океанический,Техас,США,Кондиционирование,нет,нет,27,...,63.0,1.0,1.0,0.0,0.0,28.9,0,1,мало,молодой возраст
4,8,2011.0,Лето,Cубтропический океанический,Техас,США,Кондиционирование,нет,нет,43,...,63.0,1.0,1.0,0.0,0.0,32.8,0,1,мало,молодой возраст
5,12,2011.0,Лето,Cубтропический океанический,Техас,США,Кондиционирование,нет,нет,32,...,63.0,0.0,0.0,0.0,0.0,32.8,0,1,мало,молодой возраст


In [325]:
dfcomftemp = goodtemp[['температура_воздуха_в_помещении', 'возрастная_категория']].groupby('возрастная_категория').mean().round(1)
dfcomftemp.head()
# Рассчитать среднюю комфортную температуру в зависимости от возрастной категории

,температура_воздуха_в_помещении
возрастная_категория,
молодой возраст,24.7
пожилой возраст,25.4
средний возраст,24.5


In [326]:
df3 =  goodtemp[['страна', 'пол', 'countmen']].groupby(['страна', 'пол']).count()
df3.reset_index(inplace=True)
for i in range(len(df3)):
    df3['countmen'][i] = round(df3['countmen'][i] / len(df.loc[(df['страна'] == df3['страна'][i]) & (df['пол'] == df3['пол'][i])]) * 100)

df3.head()
# Добавить столбец с процентом удовлетворенных респондентов температурой воздуха в помещении относительно всех респондентов одной страны и одного пола

C:\Users\vrost\AppData\Local\Temp\ipykernel_19100\2660006991.py:4: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df3['countmen'][i] = round(df3['countmen'][i] / len(df.loc[(df['страна'] == df3['страна'][i]) & (df['пол'] == df3['пол'][i])]) *

,страна,пол,countmen
0,Австралия,Женский,57
1,Австралия,Мужской,78
2,Индия,Женский,49
3,Индия,Мужской,57
4,США,Женский,53


In [327]:
dftypeohl = df[['способ_охлаждения', 'температура_воздуха_в_помещении', 'rh']].groupby('способ_охлаждения').median()
dftypeohl.head()
# Рассчитать медианное значение температуры и влажности для каждого типа охлаждения 

,температура_воздуха_в_помещении,rh
способ_охлаждения,,
Вентиляция,25.30,46.3
Кондиционирование,24.35,43.7
Смешанный,24.65,56.4


In [328]:

    
pivot = pd.pivot_table(df, index=['страна', 'пол', 'возрастная_категория'], values=['температура_воздуха_в_помещении', 'температура_воздуха_на_улице', 'rh'], 
                       aggfunc='mean').round(2)
pivot.head()
# Составить сводную таблицу, в которой данные будут сгруппированы по стране, полу, возрастной группе и подсчитаны средняя температура воздуха в помещении, на улице и средняя относительная влажность для каждой из этих групп.


rh  \
страна    пол     возрастная_категория          
Австралия Женский молодой возраст       61.37   
                  пожилой возраст       44.55   
                  средний возраст       61.37   
          Мужской молодой возраст       61.67   
                  пожилой возраст       64.37   

                                        температура_воздуха_в_помещении  \
страна    пол     возрастная_категория                                    
Австралия Женский молодой возраст                                 23.84   
                  пожилой возраст                                 24.90   
                  средний возраст                                 24.10   
          Мужской молодой возраст                                 24.15   
                  пожилой возраст                                 24.73   

                                        температура_воздуха_на_улице  
страна    пол     возрастная_категория                                
Австралия Женский молодой возраст                                NaN  
                  пожилой возраст                                NaN  
                  средний возраст                                NaN  
          Мужской молодой возраст                                NaN  
                  пожилой возраст                                NaN

In [329]:
df['стандр_комф_вл'] = df.apply(standr_comf_rh, axis=1)
df.head()
# Самостоятельно найти информацию по стандартной комфортной относительной влажности в помещениях (в нашем случае - офисы). Создать новый категориальный столбец, в котором будет показано, удовлетворяет ли зафиксированная относительная влажность стандарту или нет. Категории назначить самостоятельно.

,Unnamed: 0,год,время_года,климат,город,страна,способ_охлаждения,режим_при_смешанном_типе_охлаждения,способ_обогрева,возраст,...,занавески,вентилятор,окно,двери,среднемесячная_температура_на_улице,количество_рекламаций,countmen,реклам_категория,возрастная_категория,стандр_комф_вл
0,0,2011.0,Лето,Cубтропический океанический,Техас,США,Кондиционирование,нет,нет,22,...,1.0,1.0,0.0,0.0,28.9,0,1,мало,молодой возраст,OK
1,2,2011.0,Лето,Cубтропический океанический,Техас,США,Кондиционирование,нет,нет,42,...,0.0,0.0,0.0,0.0,32.8,0,1,мало,молодой возраст,not OK
2,5,2011.0,Лето,Cубтропический океанический,Техас,США,Кондиционирование,нет,нет,27,...,1.0,1.0,0.0,0.0,28.9,0,1,мало,молодой возраст,not OK
3,6,2011.0,Лето,Cубтропический океанический,Техас,США,Кондиционирование,нет,нет,32,...,1.0,1.0,0.0,0.0,32.8,0,1,мало,молодой возраст,OK
4,8,2011.0,Лето,Cубтропический океанический,Техас,США,Кондиционирование,нет,нет,43,...,1.0,1.0,0.0,0.0,32.8,0,1,мало,молодой возраст,not OK


In [330]:
cols = list(types)
colssubs = list(subs)
cols1 = []
cols2 = []
res = []
methods = []
for i in colssubs:
    for j in cols:
        statistics, pvalue, meth = corr(df[i], df[j], subs[i], types[j])
        if pvalue < 0.05 and statistics > 0.5:
            cols1.append(i)
            cols2.append(j)
            res.append(statistics)
            methods.append(meth)
dfress = pd.DataFrame()
dfress['column 1'] = cols1
dfress['column 2'] = cols2
dfress['results'] = res
dfress['methods'] = methods

dfress.sort_values('results', ascending=False).reset_index()
dfress


,column 1,column 2,results,methods
0,ощущение_температуры,утепление,55.165406,Kruskal-Wallis
1,ощущение_температуры,количество_рекламаций,65.924297,Kruskal-Wallis
2,ощущение_температуры,температура_воздуха_в_помещении,38.578075,Kruskal-Wallis
3,ощущение_температуры,rh,32.329531,Kruskal-Wallis
4,ощущение_температуры,вентилятор,52.198318,Kruskal-Wallis
5,ощущение_температуры_(bool),количество_рекламаций,14805.000000,Mann-Whitneyu
6,ощущение_температуры_(bool),температура_воздуха_в_помещении,13653.000000,Mann-Whitneyu
7,ощущение_температуры_(bool),среднемесячная_температура_на_улице,14781.500000,Mann-Whitneyu
8,предпочтительное_изменение_температуры,утепление,11.138927,Kruskal-Wallis
9,предпочтительное_изменение_температуры,количество_рекламаций,16.101239,Kruskal-Wallis
